In [1]:
# use full window width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import numpy as np
os.chdir('..')
import virl
from matplotlib import pyplot as plt

In [2]:
import pandas as pd
#import display, HTML
from IPython import display


def smooth_plot(all_rewards, smoothed_rewards,title): 
    plt.figure(2, figsize=(12, 6))
    plt.clf()
    plt.title(title)
    plt.xlabel("Epsiode") 
    plt.ylabel("Total Reward")
    plt.plot(all_rewards, '--', alpha=0.5) 
    plt.plot(smoothed_rewards) 
    plt.legend(["Rewards", "Rewards (Smoothed)"]) 
    plt.pause(0.0001)
   # display.clear_output(wait=True)

In [3]:
# Let's import basic tools for defining the function and doing the gradient-based learning
import sklearn.pipeline
import sklearn.preprocessing
#from sklearn.preprocessing import PolynomialFeatures # you can try with polynomial basis if you want (It is difficult!)
from sklearn.linear_model import SGDRegressor # this defines the SGD function
from sklearn.kernel_approximation import RBFSampler # this is the RBF function transformation method
import pandas as pd
#import display, HTML
from IPython import display


def smooth_plot(all_rewards, smoothed_rewards,title): 
    plt.figure(2, figsize=(12, 6))
    plt.clf()
    plt.title(title)
    plt.xlabel("Epsiode") 
    plt.ylabel("Total Reward")
    plt.plot(all_rewards, '--', alpha=0.5) 
    plt.plot(smoothed_rewards) 
    plt.legend(["Rewards", "Rewards (Smoothed)"]) 
    plt.pause(0.0001)
   # display.clear_output(wait=True)

In [4]:
from LinearAprxAgent import LinearAprxAgent

### Training

In [ ]:
# from plot_helper import plot
# print()
# env = virl.Epidemic(stochastic=False, noisy=False,problem_id=0)
# agent = LinearAprxAgent(env)


# #title = "Train: problem id " + str(i) + " "

# states,all_rewards, all_total_rewards,func_approximator, state_transformed, q_value = agent.train()
# intercept = func_approximator.intercept_
# coeff = func_approximator.coef_

# #plot(states,agent, all_total_rewards, all_rewards,"title")
# agent = LinearAprxAgent(env)
# states,all_rewards, all_total_rewards = agent.evaluate(intercept,coeff, state_transformed, q_value)
# plot(states,agent, all_total_rewards, all_rewards,"title")
# #func_approximator, state_transformed, q_value input to evaluate






In [ ]:
from plot_helper import plot

In [ ]:
#train on each on problem id
for i in range(0,10):
    env = virl.Epidemic(stochastic=False, noisy=False,problem_id=i)
    agent = LinearAprxAgent(env)

    title = "Train: problem id " + str(i) + " "
    print(title)
    
    states,all_rewards, all_total_rewards,func_approximator, state_transformed, q_value = agent.train()
    intercept = func_approximator.intercept_
    coeff = func_approximator.coef_

    #save for evaluation
    np.savetxt("linear_aprx_output/interceptP" + i + ".csv", intercept, delimiter=",")
    np.savetxt("linear_aprx_output/coeffP" + i + ".csv", coeff, delimiter=",")
    np.savetxt("linear_aprx_output/state_transformedP" + i + ".csv", state_transformed, delimiter=",")
    np.savetxt("linear_aprx_output/q_valueP" + i + ".csv", q_value, delimiter=",")

Decrease Factor: 0.004


In [ ]:
stochastic_noisy = {0:virl.Epidemic(stochastic=False, noisy=False),1:virl.Epidemic(stochastic=True, noisy=False),2:virl.Epidemic(stochastic=False, noisy=True),3:virl.Epidemic(stochastic=True, noisy=True)}
stochastic_noisy_print = {0:"stochastic=False, noisy=False",1:"stochastic=True, noisy=False",2:"stochastic=False, noisy=True",3:"stochastic=True, noisy=True"}

for i in range(0,4):
    
    env = stochastic_noisy[i]
    print(stochastic_noisy_print[i])
    agent = LinearAprxAgent(env)
    
    states,all_rewards, all_total_rewards,func_approximator, state_transformed, q_value = agent.train()
    intercept = func_approximator.intercept_
    coeff = func_approximator.coef_

    #save for evaluation
    np.savetxt("linear_aprx_output/interceptS" + i + ".csv", intercept, delimiter=",")
    np.savetxt("linear_aprx_output/coeffS" + i + ".csv", coeff, delimiter=",")
    np.savetxt("linear_aprx_output/state_transformedS" + i + ".csv", state_transformed, delimiter=",")
    np.savetxt("linear_aprx_output/q_valueS" + i + ".csv", q_value, delimiter=",")



In [ ]:
total_reward_sd = np.std(all_total_rewards)
total_reward_mean = np.mean(all_total_rewards)
reward_sd = np.std(all_rewards)
reward_mean = np.mean(all_rewards)

print("Total reward standard deviation = " + str(total_reward_sd))
print("Total reward mean = " + str(total_reward_mean))
print("Reward standard deviation = " + str(reward_sd))
print("Rotal reward mean = " + str(reward_mean))

In [ ]:
stochastic_noisy_print = {0:"stochastic=False, noisy=False",1:"stochastic=True, noisy=False",2:"stochastic=False, noisy=True",3:"stochastic=True, noisy=True"}
title = "Train: " + stochastic_noisy_print[0]
plot(states,agent, all_total_rewards, all_rewards,title)